In [1]:
import pandas as pd

#read csv file using pandas dataframe
df = pd.read_csv('ecommerce-session-bigquery.csv')
df = df.fillna(0) #fill the missing values with zero

#set number format, this is 
pd.set_option('display.float_format', lambda x: '{:,.0f}'.format(x) if isinstance(x, float) else x)

#check initial value of dataframe
display(df)

,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,sessionQualityDim,...,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,2515546493837534633,Organic Search,966564,Taiwan,(not set),0,0,"1,567",82,17,...,0,0,0,0,0,0,/storeitem.html,0,1,0
1,9361741997835388618,Organic Search,157377,France,not available in demo dataset,0,0,321,8,0,...,0,0,0,0,0,0,/storeitem.html,0,1,0
2,7313828956068851679,Referral,228279,United States,San Francisco,0,0,927,11,63,...,0,0,0,0,0,0,/storeitem.html,0,1,0
3,6036794406403793540,Organic Search,1615618,United States,Boulder,0,0,"1,616",13,38,...,0,0,0,0,0,0,/storeitem.html,0,1,0
4,7847280609739507227,Organic Search,37832,Canada,not available in demo dataset,0,0,"1,222",45,53,...,0,0,0,0,0,0,/storeitem.html,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8518740059623315737,Referral,83144,United States,Ann Arbor,0,0,561,18,0,...,0,0,0,0,Bags,0,/google+redesign/,0,1,0
9996,1592171014215970461,Organic Search,302378,Colombia,Bogota,0,0,365,7,0,...,0,0,0,0,Bags,0,/google+redesign/,0,1,0
9997,6776942687101397292,Affiliates,41387,Israel,not available in demo dataset,0,0,107,7,0,...,0,0,0,0,Bags,0,/google+redesign/,0,1,0
9998,3008082643543388126,Organic Search,28613,Canada,Montreal,0,0,93,11,0,...,0,0,0,0,Bags,0,/google+redesign/,0,1,0


In [2]:
#QUESTION NUMBER 1 Identify top products based on the total transaction revenue per day.

#selecting the required columns from the dataframe
selectDf = df[['date','totalTransactionRevenue','v2ProductName']]

#grouping by date and Product name to calculate sum of transactions for each product of each day
groupByDf = selectDf.groupby(["date", "v2ProductName"]).sum().reset_index()

#ranking he product based on the total revenue per day
groupByDf['ranking'] =   groupByDf.groupby('date')['totalTransactionRevenue'].rank(ascending=False)

#filtering out only those rows which have ranking as needed
topProducts = groupByDf[groupByDf['ranking'] <= 1]

#displaying result of top product per day based on total transaction revenue
display(topProducts)

,date,v2ProductName,totalTransactionRevenue,ranking
2,20160801,Chevron Shopper,"144,790,000",1
13,20160802,Chevron Shopper,"191,200,000",1
44,20160804,Google Lunch Bag,"113,850,000",1
53,20160805,Chevron Shopper,"264,780,000",1
78,20160806,Waterpoof Gear Bag,"1,157,700,000",1
...,...,...,...,...
1162,20170722,Google Leather Perforated Journal,"273,000,000",1
1166,20170726,Metal Texture Roller Pen,"67,570,000",1
1168,20170727,Metal Texture Roller Pen,"24,710,000",1
1171,20170731,Metal Texture Roller Pen,0,1


In [3]:
#QUESTION NUMBER 2 Detect any anomalies, such as a sharp decrease or increase in the number of transactions for a specific product.

#create function for zscore calculation to shows how far a data point is from the mean (average) in a dataset, measured in units of standard deviation.
def zscore(x, mean, std):
    # Parameters of z scores:
    #     x (float): The data point.
    #     mean (float): The mean of the dataset.
    #     std (float): The standard deviation of the dataset.
    return (x-mean)/std

#Select the column to be used for analysis
selectDf = df[['v2ProductName','transactions']]
grouped = selectDf.groupby('v2ProductName')['transactions']

#calculate the Z-score for the number of transactions
selectDf.insert(selectDf.columns.get_loc('transactions') + 1, 'transactionZScore', grouped.transform(lambda x: zscore(x, x.mean(), x.std())))

# Define a threshold for what constitutes an anomaly. example: Z-score above 3 or below -3 and Identify the anomalies
threshold = 3
anomalies = selectDf[(selectDf['transactionZScore'] > threshold) | (selectDf['transactionZScore'] < -threshold)]

# Display anomalies transaction
display(anomalies)


,v2ProductName,transactions,transactionZScore
334,Windup Android,1,4
337,Windup Android,1,4
338,Windup Android,1,4
339,Windup Android,1,4
340,Windup Android,1,4
...,...,...,...
9357,Waterproof Gear Bag,1,4
9386,Waterpoof Gear Bag,1,4
9406,Waterpoof Gear Bag,1,4
9466,Waterpoof Gear Bag,1,4


In [4]:
#QUESTION NUMBER 3 Identify the most profitable city or province based on the total transaction revenue.

#selecting required column
selectDf = df[['city','totalTransactionRevenue']]

#removing data from the unavailable cities name
filtered = selectDf[~selectDf['city'].isin(['(not set)', 'not available in demo dataset'])]

#group the data by city and calculate total transaction revenue
cityRevenue = filtered.groupby('city')['totalTransactionRevenue'].sum().reset_index()

#finding the maximum value of total transaction revenue by 
cityRevenue = cityRevenue.sort_values('totalTransactionRevenue', ascending=False).head(1)

#showing the result of most profitable city
print(cityRevenue)

         city  totalTransactionRevenue
103  New York           21,250,810,000
